In [1]:
import pandas as pd
import numpy as np

In [2]:
Review= pd.read_excel('C:\\Users\\Abhay\\Desktop\\DS Class Notes\\Python\\Final Project\\BankReview\\BankReviews.xlsx')

In [3]:
Review = Review.drop(['Date','BankName'], axis = 1)

In [4]:
# Key positive and negative words
Review.Reviews = Review.Reviews.str.lower()

In [5]:
def clean_splsymbols(x):
    clean_char = []
    for char in x:
        if (char.isalnum() or char == " "):
            clean_char.append(char)
    return "".join(clean_char)  


In [6]:
clean_text = Review.Reviews.apply(clean_splsymbols)
clean_text = list(clean_text)


In [7]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english',token_pattern='(?u)\\b[a-zA-Z]{2,}\\w\\w+\\b')

cv.fit(clean_text)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words='english',
                strip_accents=None,
                token_pattern='(?u)\\b[a-zA-Z]{2,}\\w\\w+\\b', tokenizer=None,
                vocabulary=None)

In [8]:
terms = cv.get_feature_names()

In [9]:
terms


['aaron',
 'aarons',
 'ability',
 'able',
 'absoltely',
 'absolute',
 'absolutely',
 'accept',
 'acceptable',
 'accepted',
 'access',
 'accessible',
 'accommodate',
 'accommodating',
 'accomplished',
 'account',
 'accountant',
 'accross',
 'accurate',
 'accurately',
 'accusing',
 'achieve',
 'acquire',
 'acquired',
 'action',
 'actual',
 'actually',
 'adam',
 'adaml',
 'adampac',
 'adan',
 'added',
 'addedconsolidationit',
 'addition',
 'additional',
 'address',
 'addressed',
 'addressresolve',
 'adjust',
 'adjusted',
 'admin',
 'admiration',
 'advertise',
 'advice',
 'advised',
 'advocated',
 'afford',
 'agai',
 'again2',
 'againthank',
 'agent',
 'agent3',
 'agents',
 'aggravating',
 'aggravation',
 'agree',
 'agreed',
 'agreement',
 'ahead',
 'airfare',
 'airline',
 'alerted',
 'alex',
 'alievatied',
 'alina',
 'allow',
 'allowed',
 'alpine',
 'altho',
 'alves',
 'amazing',
 'american',
 'amerisave',
 'amortgage',
 'amounts',
 'andd',
 'anderson',
 'andor',
 'andrea',
 'andrew',
 'a

In [10]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
si = SentimentIntensityAnalyzer()
posList = []
negList = []
newList = []

for x in terms:
    if(si.polarity_scores(x)['compound']) >= 0.1:
        posList.append(x)
    elif(si.polarity_scores(x)['compound']) <= -0.1:
         negList.append(x)
    else:
         newList.append(x)

In [11]:
pos_count = []
for word in posList:
    count=0
    for text in Review['Reviews']:
        if word in text:
            count=count+1
    pos_count.append(count)

In [12]:
posi_count = pd.Series(pos_count, name= 'count')
posi_word = pd.Series(posList, name='PositiveWords')

newpos = pd.concat([posi_word,posi_count], axis=1)

In [13]:
newpos.sort_values(by='count', ascending=False).head(10)

,PositiveWords,count
201,recommend,171
125,help,120
118,great,118
241,sure,111
247,thank,109
27,best,76
108,friend,62
81,easy,61
210,responsive,59
250,thanks,58


In [14]:
neg_count = []
for word in negList:
    count=0
    for text in Review['Reviews']:
        if word in text:
            count=count+1
    neg_count.append(count)

In [15]:
negi_count = pd.Series(neg_count, name= 'count')
negi_word = pd.Series(negList, name='PositiveWords')

newneg = pd.concat([negi_word,negi_count], axis=1)

In [16]:
newneg.sort_values(by='count', ascending=False).head(10)

,PositiveWords,count
61,hard,42
95,problem,31
114,stress,28
11,broke,24
115,stressful,24
77,lower,23
96,problems,16
83,mistake,16
111,stop,12
103,resent,11


In [ ]:
# Classification of reviews


In [18]:

def senti_analysis(text): 
    analysis = TextBlob(text) 
    if analysis.sentiment.polarity > 0: 
        return 'positive'
    elif analysis.sentiment.polarity < 0: 
        return 'negative'
    else: 
        return 'neutral'

In [20]:
from textblob import TextBlob
senti = []
for x in Review['Reviews']:
    senti.append(senti_analysis(x))

In [22]:
reviews_text = pd.Series(Review['Reviews'],name="Reviews")
review_sentiment = pd.Series(senti,name="Sentiment")
ReviewClassification = pd.concat([reviews_text,review_sentiment],axis=1)

In [23]:
ReviewClassification

,Reviews,Sentiment
0,"great job, wyndham capital! each person was pr...",positive
1,matthew richardson is professional and helpful...,positive
2,we had a past experience with wyndham mortgage...,negative
3,we have been dealing with brad thomka from the...,positive
4,i can't express how grateful i am for the supp...,positive
...,...,...
500,\r\ni never write reviews but had to this time...,positive
501,\r\nit all started when bob g ran a credit che...,positive
502,\r\nwhat a horrible experience. we have excell...,positive
503,"\r\nrep was extremely professional, friendly, ...",positive


In [ ]:
# Predict Stars

In [4]:
X_var = Review.Reviews
Y_var = Review.Stars

In [5]:
X_var

0      Great job, Wyndham Capital! Each person was pr...
1      Matthew Richardson is professional and helpful...
2      We had a past experience with Wyndham Mortgage...
3      We have been dealing with Brad Thomka from the...
4      I can't express how grateful I am for the supp...
                             ...                        
500    \r\nI never write reviews but had to this time...
501    \r\nIt all started when Bob G ran a credit che...
502    \r\nWhat a horrible experience. We have excell...
503    \r\nRep was extremely professional, friendly, ...
504    \r\nI was working with a loan consultant from ...
Name: Reviews, Length: 505, dtype: object

In [6]:
import re
corpus = []

for i in range(0, len(X_var)):
    clean = re.sub(r"[-()\"#/@;:{}`+=~|.!?,'0-9]", "", str(X_var[i]))
    clean = re.sub(r'\W', ' ', clean)   #Not words
    clean = clean.lower()
    
    corpus.append(clean)
    

In [120]:
corpus

['great job wyndham capital each person wa professional and helped u move through our refinance process smoothly thank you',
 'matthew richardson is professional and helpful he helped u find the correct product for our mortgage thank you very much for the excellent service matthew',
 'we had a past experience with wyndham mortgage and would without question use again and again if needed wyndham went beyond the extra mile to not only right a wrong encountered from the servicer we were dealing on our previous loan but they pulled together and found a viable option for u that ultimately saved u money we would highly recommend brad thomka and the wyndham capital mortgage team for your mortgage need sincerest thanks wyndham ed lind',
 'we have been dealing with brad thomka from the beginning of what started out to be a very stressful time for u but with the help from brad and the entire wyndham mortgage team it wa turned into a happy ending brad and wyndham went beyond the extra mile to not

In [93]:
import nltk
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

# Lemmatization
for i in range(0 , len(corpus)):
    words = nltk.word_tokenize(corpus[i])
    words = [lemmatizer.lemmatize(word) for word in words]
    corpus[i] = ' '.join(words)   

In [94]:
# TF-IDF
'''
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features = 2000, stop_words = stopwords.words('english'))
X = vectorizer.fit_transform(corpus).toarray()
'''

In [108]:
# Countvectorizer
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer(stop_words= stopwords.words('english'))
X = vec.fit_transform(corpus).toarray()



In [109]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [110]:
# train test split
from sklearn.model_selection import train_test_split

XTrain, XTest, YTrain , YTest = train_test_split(X,Y_var,test_size = 0.3, random_state=123)

In [97]:
YTest


372    5
310    5
444    5
421    5
429    5
      ..
91     1
150    5
320    5
471    5
493    1
Name: Stars, Length: 152, dtype: int64

In [111]:
# logistic regression
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

lr.fit(XTrain,YTrain)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [112]:
# check accuracy

from sklearn import metrics

test_y_acc = metrics.accuracy_score(YTest, lr.predict(XTest))
train_y_acc = metrics.accuracy_score(YTrain, lr.predict(XTrain))

print('test_score : ',test_y_acc)
print('train_score : ',train_y_acc )

test_score :  0.9342105263157895
train_score :  0.9943342776203966


In [119]:
# predict

pred_star = pd.DataFrame(lr.predict(X), columns = ['predected_stars'])

final = pd.concat([Review,pred_star], axis=1)
final.head(20)

,Stars,Reviews,predected_stars
0,5,"Great job, Wyndham Capital! Each person was pr...",5
1,5,Matthew Richardson is professional and helpful...,5
2,5,We had a past experience with Wyndham Mortgage...,5
3,5,We have been dealing with Brad Thomka from the...,5
4,5,I can't express how grateful I am for the supp...,5
5,5,I had the pleasure of working with Wyndham Cap...,5
6,5,My experience with Mattison was beyond greatly...,5
7,5,Patrick answered all my questions by email imm...,5
8,5,I loved working with this group of people! The...,5
9,5,Great web interface for both the loan applicat...,5
